# TensorFlow Classification

In [1]:
#import library
import pandas as pd

In [2]:
#read data
data = pd.read_csv('bdata.csv')

In [3]:
#show data
data.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,1
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,1
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,1
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,1
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,1


In [4]:
#change column name 
data = data.rename(columns={'diagnosis': 'diagnosis_numeric'})

### Normalize the dataset

In [5]:
cols_to_norm = ['radius_mean', 'area_mean']


In [6]:
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

### Create the Feature Columns to be accessed in the model

In [7]:
import tensorflow as tf

In [8]:
radius_mean = tf.feature_column.numeric_column('radius_mean')
area_mean = tf.feature_column.numeric_column('area_mean')

### Putting them together

In [9]:
feat_cols = [radius_mean, area_mean]

### Train Test Split

In [10]:
#Drop the Class column because it will be predicted by model and will be provided as a label to train test split
x_data = data.drop('diagnosis_numeric',axis=1)

In [11]:
labels = data['diagnosis_numeric']

In [12]:
from sklearn.model_selection import train_test_split

# DNN Classifier

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.33, random_state=101)

In [14]:
featureColumns = [radius_mean, area_mean]

In [15]:
inputFunction = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=1000,shuffle=True)

In [16]:
dnnClassifierModel = tf.estimator.DNNClassifier(hidden_units=[512, 256, 128],
                                                feature_columns=featureColumns,
                                                n_classes=2,
                                                activation_fn=tf.nn.tanh,
                                                optimizer=lambda: tf.train.AdamOptimizer(
                                                    learning_rate=tf.train.exponential_decay(learning_rate=0.001,
                                                    global_step=tf.train.get_global_step(),
                                                    decay_steps=1000,
                                                    decay_rate=0.96)))

dnnClassifierModel.train(input_fn=inputFunction,steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\sarah\\AppData\\Local\\Temp\\tmpwh856dqs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000195545C79C8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for 

C:\Users\sarah\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\sarah\AppData\Local\Temp\tmpwh856dqs\model.ckpt.
INFO:tensorflow:loss = 69.39322, step = 1
INFO:tensorflow:global_step/sec: 183.199
INFO:tensorflow:loss = 32.808674, step = 101 (0.548 sec)
INFO:tensorflow:global_step/sec: 174.898
INFO:tensorflow:loss = 26.907196, step = 201 (0.570 sec)
INFO:tensorflow:global_step/sec: 198.49
INFO:tensorflow:loss = 32.03012, step = 301 (0.504 sec)
INFO:tensorflow:global_step/sec: 208.73
INFO:tensor

In [17]:
dnnClassifierModel.train(input_fn=inputFunction,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sarah\AppData\Local\Temp\tmpwh856dqs\model.ckpt-1000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1000 into C:\Users\sarah\AppData\Local\Temp\tmpwh856dqs\model.ckpt.
INFO:tensorflow:loss = 28.412535, step = 1001
INFO:tensorflow:global_step/sec: 126.852
INFO:tensorflow:loss = 23.959457, step = 1101 (0.793 sec)
INFO:tensorflow:global_step/sec: 202.268
INFO:tensorflow:loss = 33.00386, step = 1201 (0.494 sec)
INFO:tensorflow:global_step/sec: 210.011
INFO:tensorflow:loss = 26.061174, step = 1301 (0.475 sec)
INFO:tensorflow:global_step/sec: 216.762
INFO:tensorflow:loss = 27.510088, step = 1401 (0.463 sec)
INFO:tensorflow:global_step/sec: 210.833


In [18]:
evaluateInputFunction = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)
dnnClassifierModel.evaluate(evaluateInputFunction)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-03-02T12:29:38Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sarah\AppData\Local\Temp\tmpwh856dqs\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-03-02-12:29:38
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.87234044, accuracy_baseline = 0.62234044, auc = 0.9219333, auc_precision_recall = 0.9148743, average_loss = 0.31742874, global_step = 2000, label/mean = 0.3776596, loss = 3.140874, precision = 0.8852459, prediction/mean = 0.3753569, recall = 0.7605634
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: C:\Users\sarah\AppData\Local\Temp\tmpwh856dqs\model.ckpt-2000


{'accuracy': 0.87234044,
 'accuracy_baseline': 0.62234044,
 'auc': 0.9219333,
 'auc_precision_recall': 0.9148743,
 'average_loss': 0.31742874,
 'label/mean': 0.3776596,
 'loss': 3.140874,
 'precision': 0.8852459,
 'prediction/mean': 0.3753569,
 'recall': 0.7605634,
 'global_step': 2000}